### Historical Product Demand - Analysis & Models
**Data Source:** [Kaggle](https://www.kaggle.com/felixzhao/productdemandforecasting)

**Data Description from Kaggle:** *The dataset contains historical product demand for a manufacturing company with footprints globally. The company provides thousands of products within dozens of product categories. There are four central warehouses to ship products within the region it is responsible for. Since the products are manufactured in different locations all over the world, it normally takes more than one month to ship products via ocean to different central warehouses. If forecasts for each product in different central with reasonable accuracy for the monthly demand for month after next can be achieved, it would be beneficial to the company in multiple ways.*

**Objective:** *Is it possible to make forecasts for thousands of products (some of them are highly variable in terms of monthly demand) for the the month after next?* <br/>
The latest data month is Jan 2017, thus forecast is for Mar 2017 (onwards). <br/>

RMSE is used as performance metric in this forecast.

**Assumptions:** <br/>
A1/ Date refers to shipping date, not order date. Otherwise, forecast cannot be done. Shipping date indicates the time when products need to be avaiable at these warehouse. <br/>
A2/ Order demand refers to the order quantity by customers (actual demands), not the order quantity that can be fulfilled by warehouses. <br/>
Other assumptions will be made throughout the analysis. 

In [63]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from operator import attrgetter
from statsmodels.tsa.arima.model import ARIMA
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from prophet import Prophet

# For auto ARIMA functionality
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMAResults
import itertools

In [69]:
# Read data
data = pd.read_csv("data_purchases_2.csv")
data.dtypes
print(data.columns)


Index(['id', 'item_id', 'date', 'quantity', 'cost (CLP)', 'delivery_date'], dtype='object')


1/ All columns are formatted as characters and need to be reformatted: <br/> 
-**Date** to datetime. <br/>
-**Order_Demand** to numeric. Order_Demand values include negative numbers that are formated as "(number)". These values need to be reformatted as "-number" before being converted to numeric.

2/ Warehouse and category information do not contribute to forecasting and will be removed. <br/>
- **Warehouse:** As per my understanding from data description, the four central warehouses serve demands for the same region. Therefore, the forecast will be performed for total demands in the region, without consideration of warehouse information. Product quantity per warehouse will be calculated based on other information like warehouse capacity and transportation cost, which is not in the scope of this demand forecasting. <br/>
- **Product_Category:** There is no other information to understand about product characteristics in each category. Meanwhile, the number of products per category is high, which makes visualization on data patterns of products per category impossible. Thus, it is difficult to understand products' data patterns in accordance with category number. Category column does not well explain demands in this dataset, and thus can be dropped.

In [70]:
# Format Date to datetime
data['date'] = pd.to_datetime(data['date'])


# Drop warehouse and category columns
data = data.drop(columns=['id'])
data = data.drop(columns=['cost (CLP)'])

# Sort data by period
data = data.sort_values('date').reset_index().drop('index',axis=1)
print(data.columns)

Index(['item_id', 'date', 'quantity', 'delivery_date'], dtype='object')


In [71]:
print("Data Types")
print(data.dtypes)
print("\t")

print(data.head())
print("\t")

print("Data Dimension",data.shape)
print("\t")

print("The number of products is",len(data['item_id'].value_counts().index))
print("Period range is from",data['date'].min(),"to", data['date'].max())
print("Order Qty is from",data['quantity'].min(),"to", data['quantity'].max())
print("\t")

print('Number of missing values by column',[sum(data[i].isnull()) for i in data.columns])
print('All missing values are in Date column.')

Data Types
item_id                   int64
date             datetime64[ns]
quantity                  int64
delivery_date            object
dtype: object
	
   item_id       date  quantity delivery_date
0     1810 2020-01-13         3     1/25/2020
1     1804 2020-01-13         2     1/25/2020
2     1856 2020-01-13         8     1/24/2020
3     1803 2020-01-13         5     1/28/2020
4     1815 2020-01-13         2     1/23/2020
	
Data Dimension (8517, 4)
	
The number of products is 81
Period range is from 2020-01-13 00:00:00 to 2024-03-04 00:00:00
Order Qty is from 1 to 151
	
Number of missing values by column [0, 0, 0, 0]
All missing values are in Date column.


##### Aggregate data by month
Forecasting is performed at monthly horizons, thus the dataset should first be aggregated by month. Date is extracted with Month & Year only.

In [72]:
data['date'] = data['date'].dt.to_period('M')
data = data.rename(columns = {"date": 'Period'})
data = data.groupby(['item_id','Period'])['quantity'].sum().reset_index().sort_values('Period'
            ).reset_index().drop('index',axis=1)
data.head()

,item_id,Period,quantity
0,1803,2020-01,5
1,1814,2020-01,16
2,1815,2020-01,17
3,1856,2020-01,8
4,1810,2020-01,5


In [57]:
# Check to see if periods in dataset is continuous
# Create a duration with continuous periods
full_period = pd.date_range('2020-01-01','2023-12-31', freq='MS').to_period('M')
full_period = set(full_period)
data_period = set(data['Period'])
full_period.difference(data_period)
# The missing periods are 5 months in 2011, including Feb, Mar, Apr, Jul, and Aug.
# There are various possible reasons for the missing periods: No demands are in these months, warehouses to be
# closed in these months for some reason, missing data in these periods, etc.
# To ensure that the training data will not be misleading, all data before Sep 2011 will be removed.
#data = data.loc[data['Period'] > '2011-08']

set()

##### Check to see which products are eligible for forecasting

Some analysis is done to see if any products should be excluded from the forecasting. <br/>
Several criteria are as below: <br/>
**1/** All data in Jan, 2017 should be removed; otherwise, model interpretation will be misled. <br/>
Reason: The latest date in this dataset is 2017/01/09 while forecasting is for monthly horizon. <br/>
**2/** Products that have no order quantities in 2016 will be removed. <br/>
Reason: It is highly likely that the products have been stopped already and will have no future demand. The duration of 1 year helps cover the cases of seasonal products.  <br/>
**3/** Products with demand history less than 24 months will be removed. <br/>
Reason: A minimum of 2 years' data is required to forecast trends and seasonality using statistical forecasting methods. Also, in cases when history is too short (6 months for example), the products are likely to be new products and forecasting methods for new products are different. 

In [13]:
# # Criteria 1: Remove data in Jan, 2017
# data = data.loc[data['Period']<'2017-01']

In [14]:
# # Criteria 2: Remove stopped products
# latest_datamonth = data.groupby('Product_Code')['Period'].max().reset_index()
# latest_datamonth = latest_datamonth.loc[latest_datamonth['Period'] > '2015-12']
# data = data.loc[data['Product_Code'].isin(latest_datamonth['Product_Code'])]

In [15]:
# # Criteria 3: Remove new products
# duration_data = data.groupby('Product_Code').agg({'Period': ['min', 'max']}).reset_index()
# duration_data['Duration'] = (duration_data[('Period', 'max')] - duration_data[('Period', 'min')]
#                             ).apply(attrgetter('n')) + 1
# duration_data = duration_data.loc[duration_data['Duration'] > 24 ]
# data = data.loc[data['Product_Code'].isin(duration_data['Product_Code'])]

##### Construct time series in a columnar format

In [58]:
data = pd.pivot_table(data, values = 'quantity', index = 'Period', columns = 'item_id',aggfunc=np.sum
                     ).reset_index().rename_axis("", axis="columns")

#Fill in missing values with 0. Months with missing values are implied to have zero demands.
data = data.fillna(0)
data = data.set_index('Period')
data.head()

C:\Users\56979\AppData\Local\Temp\ipykernel_25356\2208093037.py:1: FutureWarning: The provided callable <function sum at 0x000001F8FF8334C0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data = pd.pivot_table(data, values = 'quantity', index = 'Period', columns = 'item_id',aggfunc=np.sum


,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,...,1856,1857,1858,1859,1860,1861,1863,1866,1867,1869
Period,,,,,,,,,,,,,,,,,,,,,
2020-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02,37.0,50.0,12.0,14.0,20.0,40.0,17.0,25.0,37.0,20.0,...,14.0,0.0,2.0,8.0,0.0,0.0,0.0,6.0,0.0,0.0
2020-03,109.0,64.0,27.0,67.0,108.0,162.0,30.0,56.0,47.0,55.0,...,9.0,0.0,0.0,0.0,8.0,3.0,9.0,2.0,6.0,6.0
2020-04,54.0,52.0,13.0,70.0,38.0,101.0,28.0,31.0,42.0,40.0,...,13.0,0.0,0.0,0.0,19.0,23.0,17.0,2.0,25.0,11.0
2020-05,66.0,55.0,21.0,85.0,85.0,117.0,23.0,38.0,41.0,50.0,...,5.0,17.0,2.0,0.0,25.0,21.0,15.0,2.0,28.0,15.0


In [17]:
# # The dataset includes data for 2061 products, which takes Python too long to return model results.
# # Thus, I sampled only several products with different patterns for demonstration purpose
# data = data[['Product_0002','Product_0138','Product_0597','Product_0875','Product_2066',
#                'Product_2091','Product_2127','Product_2165']]
# data.shape

(64, 8)

##### Try with models
Forecasting methods: Each product will be tested with different models to find out the models which suit with products' data patterns the most. Performance will be compared using RMSE. After the best model is picked, product forecast will be created by the model trained on the most recent data values.

In [73]:
# function to split train and test set
# history length of a product starts from the month of first order, not all products have the same history length
# the function is to split train-test based on product's history length instead of dataset length
def train_test(data):
    myList = data.tolist()
    i = myList.index(next(filter(lambda x: x!=0, myList)))
    data = data.iloc[i:,]
    train = data[:int(0.7*(len(data)))]
    test = data[int(0.7*len(data)):]
    return train, test, data

# create data for forecasting
start = data.index.tolist()[-1] + 3
fcastperiods = 12  # forecast periods is subject to change by forecast users
full_period = [start + x for x in range(0,fcastperiods)]


<bound method NDFrame.head of       item_id   Period  quantity
0        1803  2020-01         5
1        1814  2020-01        16
2        1815  2020-01        17
3        1856  2020-01         8
4        1810  2020-01         5
...       ...      ...       ...
3505     1829  2024-03        13
3506     1840  2024-03         4
3507     1856  2024-03         2
3508     1869  2024-03        13
3509     1833  2024-03         2

[3510 rows x 3 columns]>


##### ARIMA Model

In [1]:
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import warnings

def arimafcast(data, fcastperiods=12):
    warnings.filterwarnings("ignore")  # Suppress warnings
    Arima = ['Arima']
    ArimaFcastPerf = pd.DataFrame({'Models': Arima})
    
    full_period = pd.date_range(start=data.index.max() + pd.DateOffset(months=1), 
                                periods=fcastperiods, freq='M')
    ArimaData = pd.DataFrame({'Period': full_period, 'Model': 'Arima'})
    
    def find_order(series):
        # Simple function to determine order based on ADF test
        d = adfuller(series)[1] > 0.05  # If p-value > 0.05, d=1, else d=0
        return (1, int(d), 1)  # Simple (1,d,1) model
    
    for i in data.columns:
        try:
            train, test, full = train_test(data[i])
            
            # Test model
            order = find_order(train)
            model_pred = ARIMA(train, order=order)
            results_pred = model_pred.fit()
            pred = results_pred.forecast(steps=len(test))
            ArimaFcastPerf[i] = sqrt(mean_squared_error(test, pred))
            
            # Forecast model
            order = find_order(full)
            model_fc = ARIMA(full, order=order)
            results_fc = model_fc.fit()
            forecast = results_fc.forecast(steps=fcastperiods)
            ArimaData[i] = np.round(forecast)
        except Exception as e:
            print(f"Error processing column {i}: {str(e)}")
            ArimaFcastPerf[i] = np.nan
            ArimaData[i] = np.nan
            
    return ArimaFcastPerf, ArimaData

ModuleNotFoundError: No module named 'numpy'

##### SARIMA Model

In [20]:
%%capture 

# build function to run model for all columns
SArima = ['SArima']
SArimaFcastPerf = pd.DataFrame({'Models': SArima})
SArimaData = pd.DataFrame({'Period': full_period, 'Model': 'SArima'})

def sarimafcast(data):
    for i in data.columns:
        try:
            train, test, full = train_test(data[i])
            
            # Test model
            model_pred = auto_arima(train, start_p=0, start_q=0, max_p=3, max_q=3,
                                  m=12, trace=True, 
                                  error_action='ignore', suppress_warnings=True)
            model_pred.fit(train)
            pred = np.round(model_pred.predict(n_periods=len(test)))
            SArimaFcastPerf[i] = sqrt(mean_squared_error(test,pred))
            
            # Forecast model
            model_fc = auto_arima(full, start_p=0, start_q=0, max_p=3, max_q=3,
                                  m=12, trace=True, 
                                  error_action='ignore', suppress_warnings=True)
            model_fc.fit(full)
            forecast = np.round(model_fc.predict(n_periods=fcastperiods+2))
            SArimaData[i] = forecast[-fcastperiods:]
        except:
            SArimaFcastPerf[i] = np.nan
            SArimaData[i] = np.nan
    return SArimaFcastPerf, SArimaData
sarimafcast(data)

##### Simple Exponential Smoothing

In [21]:
%%capture

# build function to run model for all columns
SES = ['SES']
SESFcastPerf = pd.DataFrame({'Models': SES})
SESData = pd.DataFrame({'Period': full_period, 'Model': 'SES'})

def sesfcast(data):
    for i in data.columns:
        try:
            train, test, full = train_test(data[i])
                
            # Test model
            model_pred = SimpleExpSmoothing(train).fit()
            pred = np.round(model_pred.forecast(len(test)))
            SESFcastPerf[i] = sqrt(mean_squared_error(test,pred))
                
            # Forecast model
            model_fc = SimpleExpSmoothing(full).fit()
            forecast = np.round(model_fc.forecast(fcastperiods+2))
            SESData[i] = forecast.tolist()[-fcastperiods:]            
        except:
            SESFcastPerf[i] = np.nan
            SESData[i] = np.nan
    return SESFcastPerf, SESData
sesfcast(data)

##### Double Exponential Smoothing
Note: Double & Triple Exponential Smoothing only apply to positive data values

In [22]:
%%capture

# build function to run model for all columns
DES = ['DES']
DESFcastPerf = pd.DataFrame({'Models': DES})
DESData = pd.DataFrame({'Period': full_period, 'Model': 'DES'})

def desfcast(data):
    for i in data.columns:
        try:
            train, test, full = train_test(data[i])
            if (train.min() != 0) and (test.min() !=0):
                # Test model
                model_pred = Holt(train,damped=True).fit()
                pred = np.round(model_pred.forecast(len(test)))
                DESFcastPerf[i] = sqrt(mean_squared_error(test,pred))
                    
                # Forecast model
                model_fc = Holt(full,damped=True).fit()
                forecast = np.round(model_fc.forecast(fcastperiods+2))
                DESData[i] = forecast.tolist()[-fcastperiods:]
            else:
                DESFcastPerf[i] = np.nan
                DESData[i] = np.nan  
        except:
            DESFcastPerf[i] = np.nan
            DESData[i] = np.nan
    return DESFcastPerf, DESData
desfcast(data)

##### Triple Exponential Smoothing

In [23]:
%%capture

# build function to run model for all columns
TES = ['TES']
TESFcastPerf = pd.DataFrame({'Models': TES})
TESData = pd.DataFrame({'Period': full_period, 'Model': 'TES'})

def tesfcast(data):
    for i in data.columns:
        try:
            train, test, full = train_test(data[i])
            if (train.min() != 0) and (test.min() != 0):
                # Test model                
                model_pred = ExponentialSmoothing(train, seasonal_periods=12, 
                                                 trend='add', seasonal='mul', damped=True
                                                ).fit(optimized=True,use_boxcox=True)
                pred = np.round(model_pred.forecast(len(test)))
                TESFcastPerf[i] = sqrt(mean_squared_error(test,pred))
                    
                # Forecast model
                model_fc = ExponentialSmoothing(full, seasonal_periods=12, 
                                                 trend='add', seasonal='mul', damped=True
                                                ).fit(optimized=True,use_boxcox=True)
                forecast = np.round(model_fc.forecast(fcastperiods+2))                
                TESData[i] = forecast.tolist()[-fcastperiods:]
            else:
                TESFcastPerf[i] = np.nan
                TESData[i] = np.nan       
        except:
            TESFcastPerf[i] = np.nan
            TESData[i] = np.nan   
    return TESFcastPerf, TESData
tesfcast(data)

##### Prophet

In [24]:
%%capture

# build function to run model for all columns
Prophetf = ['Prophet']
ProphetfFcastPerf = pd.DataFrame({'Models': Prophetf})
ProphetfData = pd.DataFrame({'Period': full_period, 'Model': 'Prophet'})
    
def prophetfForecast(data): 
    for i in data.columns:
        try:
            myList = data[i].tolist()
            j = myList.index(next(filter(lambda x: x!=0, myList)))
            temp = data.iloc[j:,]        
            temp['ds']=temp.index.astype('datetime64[ns]')
            temp = temp.reset_index(drop=True)
            temp = temp.rename(columns = {i:'y'})
            temp = temp.loc[:,['ds','y']]
                
            train = temp[:int(2/3*(len(temp)))]
            test = temp[int(2/3*len(temp)):]
               
            # Test model
            model_pred = Prophet()
            model_pred.fit(train)
            pred = model_pred.predict(test)
            ProphetfFcastPerf[i] = sqrt(mean_squared_error(test['y'].values,pred['yhat'].values))
                
            # Forecast model
            model_fc = Prophet()
            model_fc.fit(temp)
                        
            fc_ds = ProphetfData.copy().set_index('Period')
            fc_ds['ds']=fc_ds.index.astype('datetime64[ns]')
            fc_ds = fc_ds.reset_index(drop=True).drop('Model',axis=1)
                
            forecast = model_fc.predict(fc_ds)
            ProphetfData[i] = np.round(forecast['yhat'].values)
        except:
            ProphetfFcastPerf[i] = np.nan
            ProphetfData[i] = np.nan
    return ProphetfFcastPerf, ProphetfData    
prophetfForecast(data)

##### Compare and select the best model for each product

In [25]:
#Combine all performance tables
FcastPerf = pd.concat([ArimaFcastPerf,SArimaFcastPerf,SESFcastPerf,DESFcastPerf,TESFcastPerf,ProphetfFcastPerf]
                     ).set_index('Models')
FcastPerf

,Product_0002,Product_0138,Product_0597,Product_0875,Product_2066,Product_2091,Product_2127,Product_2165
Models,,,,,,,,
Arima,118217.407066,1920.982657,4.339739,408.088226,239.215384,102.328280,165.067495,83.539779
SArima,274631.574246,1920.982657,4.339739,559.418493,NaN,102.328280,165.067495,83.539779
SES,158304.234195,2229.192785,2.886751,544.968806,231.796737,102.033554,165.067495,83.539779
DES,158599.354591,1536.538454,NaN,NaN,NaN,NaN,NaN,NaN
TES,192662.202227,1306.719376,NaN,NaN,NaN,NaN,NaN,NaN
Prophet,319742.296789,4842.383948,5.466904,960.321731,212.299737,206.114540,471.901199,151.607379


In [26]:
# Find best model for each product
Model = pd.DataFrame()

for i in FcastPerf.columns:
    # Find the one with lowest RMSE. Choose the first one in case of more than 1 min values.
    Model[i] = list([FcastPerf.loc[FcastPerf[i] == FcastPerf[i].min()].index[0]])
Model

,Product_0002,Product_0138,Product_0597,Product_0875,Product_2066,Product_2091,Product_2127,Product_2165
0,Arima,TES,SES,Arima,Prophet,SES,Arima,Arima


In [27]:
# Filter list of products per model
model_dict = {}
model_list = ['Arima', 'SArima', 'SES', 'DES', 'TES', 'Prophet']
for i in model_list:
    model_dict[i] = []

for i in Model.columns:
    for j in model_list:
        if any(Model[i] == j):
            model_dict[j].append(i)
model_dict

{'Arima': ['Product_0002', 'Product_0875', 'Product_2127', 'Product_2165'],
 'SArima': [],
 'SES': ['Product_0597', 'Product_2091'],
 'DES': [],
 'TES': ['Product_0138'],
 'Prophet': ['Product_2066']}

In [28]:
# Extract forecast from equivalent models to the final forecast dataframe
AllForecast = pd.concat([ArimaData, SArimaData, SESData, DESData, TESData, ProphetfData])
FinalForecast = pd.DataFrame({'Period': full_period})

for i in model_list:
    for j in model_dict[i]:
        FinalForecast[j] = AllForecast.loc[AllForecast['Model'] == i][j]
FinalForecast = FinalForecast.set_index('Period')

In [29]:
# Combine historical and forecast data. Insert Data_Type column for filter purpose and move it to front.
HistoricalData = data[sorted(data)]
FinalForecast = FinalForecast[sorted(FinalForecast)]
FinalData = pd.concat([HistoricalData,FinalForecast])
    
FinalData['Data Type'] = np.nan
FinalData['Data Type'].iloc[:-fcastperiods] = 'Historical Data'
FinalData['Data Type'].iloc[-fcastperiods:] = 'Forecast'
co_list = FinalData.columns.tolist() 
co_list.insert(0, co_list.pop(co_list.index('Data Type'))) 
FinalData = FinalData.reindex(columns= co_list)    

C:\Users\ntkcu\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [30]:
# Save final data
#FinalData.to_csv("FinalData.csv")
FinalData

,Data Type,Product_0002,Product_0138,Product_0597,Product_0875,Product_2066,Product_2091,Product_2127,Product_2165
Period,,,,,,,,,
2011-09,Historical Data,0.0,0.0,0.0,5450.0,0.0,0.0,0.0,0.0
2011-10,Historical Data,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-11,Historical Data,0.0,2040.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-12,Historical Data,221000.0,5812.0,0.0,0.0,0.0,0.0,0.0,107.0
2012-01,Historical Data,65000.0,1590.0,0.0,0.0,0.0,0.0,0.0,115.0
...,...,...,...,...,...,...,...,...,...
2017-10,Forecast,131138.0,3939.0,2.0,261.0,90.0,98.0,132.0,91.0
2017-11,Forecast,131138.0,2116.0,2.0,261.0,214.0,98.0,132.0,91.0
2017-12,Forecast,131138.0,3541.0,2.0,261.0,110.0,98.0,132.0,91.0


In [ ]:
# Plot each product column in the final data   
for i in FinalData.columns[1:]:
    ax = FinalData[i].iloc[:-fcastperiods].plot(figsize=(15, 5), color ='green')
    FinalData[i].iloc[-fcastperiods:].plot(ax=ax, color ='red')
    fig = ax.get_figure()
    plt.show(block=False)
    plt.close(fig)

**Limitations of this forecast:** <br/>
1/ *Limited number of models:* It seems that data patterns of the products at this company vary alot. The models built could not capture all data patterns and return meaningful forecasts. <br/>
2/ *No external factors explained:* The forecast is based on historical data only and doesnt incorporate any external information like economic conditions, product price policy changes, future promotions, etc. <br/>
3/ *No consideration of returns impact:* Order returns can make negative impact on future demand. For example, customers may leave, which leads to a drop in future demand. However, there is not information to make any assumptions about this impact in this forecasting. <br/>
4/ *Omitted missing values:* It's impossible to measure the impact of removing missing values in this dataset. <br/> 